In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/College/6th sem/DA/DA Mini Project')

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
import Models 

Using TensorFlow backend.


In [5]:
# load packages
import pandas as pd
import gc
import tensorflow as tf
import numpy as np
import keras
# from keras import backend as K
# from keras.models import load_model, Model
# from keras.layers import Flatten, Dense, Dropout, Activation, Input, LSTM, Reshape, Conv2D, MaxPooling2D
# from keras.optimizers import Adam
# from keras.layers.advanced_activations import LeakyReLU

from keras.utils import np_utils
import matplotlib.pyplot as plt

# set random seeds
np.random.seed(1)
# tf.random.set_seed(2)


In [6]:
def prepare_x(data):
    df1 = data[:40, :].T
    return np.array(df1)

def get_label(data):
    lob = data[-5:, :].T
    return lob

def data_classification(X, Y, T):
    [N, D] = X.shape
    df = np.array(X)

    dY = np.array(Y)

    dataY = dY[T - 1:N]

    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]

    dataX = dataX.swapaxes(1,2)

    return dataX, dataY

In [7]:
dec_train = np.loadtxt('/content/drive/MyDrive/College/6th sem/DA/DA Mini Project/1.NoAuction_Zscore/NoAuction_Zscore_Training/Train_Dst_NoAuction_ZScore_CF_7.txt')
dec_test3 = np.loadtxt('/content/drive/MyDrive/College/6th sem/DA/DA Mini Project/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_7.txt')
dec_test4 = np.loadtxt('/content/drive/MyDrive/College/6th sem/DA/DA Mini Project/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_8.txt')
dec_test5 = np.loadtxt('/content/drive/MyDrive/College/6th sem/DA/DA Mini Project/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_9.txt')
dec_test = np.hstack(( dec_test3, dec_test4, dec_test5))
del dec_test3
del dec_test4
del dec_test5
gc.collect()

# extract limit order book data from the FI-2010 dataset
train_lob = prepare_x(dec_train)
test_lob = prepare_x(dec_test)

# extract label from the FI-2010 dataset
train_label = get_label(dec_train)
test_label = get_label(dec_test)

# prepare training data. We feed past 100 observations into our algorithms and choose the prediction horizon. 
trainX_CNN, trainY_CNN = data_classification(train_lob, train_label, T=50)
trainY_CNN = trainY_CNN[:,3] - 1
trainY_CNN = np_utils.to_categorical(trainY_CNN, 3)

# prepare test data.
testX_CNN, testY_CNN = data_classification(test_lob, test_label, T=50)
testY_CNN = testY_CNN[:,3] - 1
testY_CNN = np_utils.to_categorical(testY_CNN, 3)

In [8]:
template = [[40,50], [60,10],[120, 5], [3,1]]

In [9]:
projection_regularizer = None
projection_constraint = keras.constraints.max_norm(3.0,axis=0)
attention_regularizer = None
attention_constraint = keras.constraints.max_norm(5.0, axis=1)
dropout = 0.1

In [10]:
model = Models.TABL(template, dropout, projection_regularizer, projection_constraint, attention_regularizer, attention_constraint)
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
(None, 40, 50)
(None, 60, 10)
(None, 60, 10)
(None, 120, 5)
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 50)            0         
_________________________________________________________________
bl_1 (BL)                    (None, 60, 10)            3500      
_________________________________________________________________
activation_1 (Activation)    (None, 60, 10)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 10)            0         
_________________________________________________________________
bl_2 (BL)                    (None, 120, 5)            7850      
_________________________________________________________________
activation_2 (Activation)    (None, 120, 5)       

In [11]:
# create class weight
class_weight = {0 : 1e6/300.0,
                1 : 1e6/400.0,
                2 : 1e6/300.0}


In [12]:
# training
model.fit(trainX_CNN, trainY_CNN, batch_size=256, epochs=100, class_weight=class_weight, validation_data=(testX_CNN, testY_CNN) )


Train on 254701 samples, validate on 139538 samples
Epoch 1/100
254701/254701 [==============================] - 12s 46us/step - loss: 2867.9131 - acc: 0.5417 - val_loss: 0.7964 - val_acc: 0.6593
Epoch 2/100
254701/254701 [==============================] - 10s 38us/step - loss: 2462.8465 - acc: 0.6440 - val_loss: 0.7235 - val_acc: 0.7044
Epoch 3/100
254701/254701 [==============================] - 10s 38us/step - loss: 2350.6851 - acc: 0.6680 - val_loss: 0.7224 - val_acc: 0.7071
Epoch 4/100
254701/254701 [==============================] - 10s 38us/step - loss: 2284.2458 - acc: 0.6803 - val_loss: 0.6957 - val_acc: 0.7196
Epoch 5/100
254701/254701 [==============================] - 10s 39us/step - loss: 2243.7286 - acc: 0.6885 - val_loss: 0.7088 - val_acc: 0.7161
Epoch 6/100
254701/254701 [==============================] - 10s 39us/step - loss: 2222.0872 - acc: 0.6910 - val_loss: 0.6597 - val_acc: 0.7388
Epoch 7/100
254701/254701 [==============================] - 10s 38us/step - loss: 

In [13]:
from sklearn.metrics import classification_report

y_pred = model.predict(testX_CNN, batch_size=256, verbose=2)
y_pred_bool = np.argmax(y_pred, axis=1)

round_testy = np.argmax(testY_CNN, axis=1)

print(classification_report(round_testy, y_pred_bool))


              precision    recall  f1-score   support

           0       0.68      0.73      0.70     38439
           1       0.83      0.78      0.80     66000
           2       0.68      0.70      0.69     35099

    accuracy                           0.74    139538
   macro avg       0.73      0.73      0.73    139538
weighted avg       0.75      0.74      0.75    139538

